# Tutorial: How to impedance match an rf antenna

In [ ]:
import numpy as np
from pathlib import Path
import skrf as rf
import spin_tools as sp
import matplotlib.pyplot as plt

%matplotlib notebook
%load_ext autoreload
%autoreload 2

Load a keystone file (.s1p) of an unmatched antenna which is created by a vector network analyzer (VNA). It describes the frequency dependent response to a rf signal. Note: pathname string should be a python raw string.

In [ ]:
raw_antenna_file = Path(r"example_antenna.s1p")


# define a network object
raw_antenna_ntw = rf.Network(str(raw_antenna_file))
frequency = raw_antenna_ntw.frequency

# define the standard 50 Ohm cable which is used to connect to the antenna
Z_0 = 50
beta = frequency.w/rf.c
line = rf.DefinedGammaZ0(frequency=frequency, gamma=1j*beta, z0=Z_0)


Overview over the measured raw antenna:

In [ ]:
raw_antenna_ntw

Reduce the number of data points to increase optimization speed.
Select `f_start`and `f_end` such they are in the bounds of the frequency range of `raw_antenna_ntw`.
Select `resolution` as desired. Lower values increase calculation speed, the result may be less accurate though.

In [ ]:
f_start = 8 # MHz
f_end = 12 # MHz
resolution = 201

# reduced frequency object
frequency_reduced = rf.Frequency(f_start, f_end, 201, "MHz")

# reduce network
raw_ntw_reduced = raw_antenna_ntw.interpolate_from_f(frequency_reduced)


## Create a matching network

In this step, you will create a function that contains the blueprint for a impedance matching network.
A matching network function is structured as follows:

In [ ]:
# the signature of the function is always the same, except for the name.
def matching_network_example(line_, ntw_antenna, components):
    # components is a tuple that contains the numerical values of the components of
    # the circuit (capacitors and inductors). You can have as many components as you like,
    # see other examples.
    C1, C2 = components 

    # a small workaround to keep consistent structure, always include this line unmodified.
    n = line_.resistor(1e-6)

    # now, the actual structure of your matching network. The '**' operator appends components to the network,
    # starting furthest from the antenna, so therefore closest to the rf power source.
    # Available components are:
        ## capacitor in series with the antenna: line_.capacitor()
        ## inductor in series with the antenna: line_.inductor()
        ## capacitor parallel to the antenna: line_.shunt_capacitor()
        ## inductor parallel to the antenna: line_.shunt_inductor()
    # As a parameter, each component gets one of the named components we extracted from the tuple above.
    # Use *1e-12 for capacitors to get pF units, and *1e-9 for inductors to get nH units.

    n = n**line_.shunt_capacitor(C1*1e-12)
    n = n**line_.capacitor(C2*1e-12)

    # Finally, the antenna is added to the network
    n = n**ntw_antenna
    return n

Matching networks with 2 components are the simplest to implement in practice, but you can also simulate circtuis with more components. Like this:

In [ ]:
def matching_network_4th_order_example(line_, ntw_antenna, components):
    C1, C2, L1, L2 = components
    n = line_.resistor(1e-6)
    n = n**line_.capacitor(C1*1e-12)
    n = n**line_.shunt_inductor(L1*1e-9)
    n = n**line_.capacitor(C2*1e-12)
    n = n**line_.shunt_inductor(L2*1e-9)
    
    n = n**ntw_antenna
    return n

# Actually optimizing the matching network
Here I have defined many of the possible matching network configurations that have only 2 components. Of course, you can replace these with your own networks.

In [ ]:
def matching_network_CC_1(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    n = n**line_.shunt_capacitor(C2*1e-12)
    n = n**line_.capacitor(C1*1e-12)
    
    n = n**ntw_antenna
    return n

def matching_network_CC_2(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    n = n**line_.capacitor(C1*1e-12)
    n = n**line_.shunt_capacitor(C2*1e-12)
    
    n = n**ntw_antenna
    return n

def matching_network_1(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    
    n = n**line_.capacitor(C1*1e-12)
    n = n**line_.shunt_inductor(C2*1e-9)
    
    n = n**ntw_antenna
    return n

def matching_network_4(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    
    n = n**line_.shunt_capacitor(C1*1e-12)
    n = n**line_.inductor(C2*1e-9)
    
    n = n**ntw_antenna
    return n

def matching_network_2(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    
    n = n**line_.shunt_inductor(C2*1e-9) 
    n = n**line_.capacitor(C1*1e-12)
    
    n = n**ntw_antenna
    return n

def matching_network_3(line_, ntw_antenna, components):
    C1, C2 = components
    n = line_.resistor(1e-6)
    
    n = n**line_.inductor(C2*1e-9) 
    n = n**line_.shunt_capacitor(C1*1e-12)
    
    n = n**ntw_antenna
    return n
    

## Optimization

In [ ]:
### define parameters:

# frequency range in which the antenna should be matched
f_mask_start = 10.3e6
f_mask_stop = 10.6e6

f_mask = (f_mask_start, f_mask_stop)

# List of matching network functions which you want to optimize.
# All functions have to have the same number of components.

matching_networks = [matching_network_1,
                    matching_network_2,
                    matching_network_3,
                    matching_network_4]

# define bounds for component values
# for each component, a tuple containing lower and upper bound is required
bounds = [
    (1,5000), # component 1
    (1,5000)  # component 2
]

In [ ]:
results = sp.optimize_antenna(raw_ntw_reduced,
                              bounds,
                              matching_networks,
                              f_mask)

## Plot the results
If for a matching network any of the component values hit a value we set for the lower or upper bound earlier, the result will not be the best possible result. In this case, either try to set larger bounds for the component values, increase the resolution of the simulation or consider a different matching network configuration.

In [ ]:
fig_1, ax = plt.subplots(1,1)
for matching_network, res in zip(matching_networks, results):
    return_loss = matching_network(line, raw_antenna_ntw, res.x).s_db.reshape(-1)
    ax.plot(raw_antenna_ntw.frequency.f*1e-6, return_loss, label=f"{matching_network.__name__}, comp.: {np.around(res.x,2)}")
ax.legend()
plt.show()